# DAR in each brain region for each subclass

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(repr)
    library(patchwork)
    library(ggplot2)
    library(Signac)
    library(tidyverse)
    library(GenomicRanges)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(scater)
    library(ggrepel)
    library(fs)
})
options(future.globals.maxSize = Inf)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
setwd("/tscc/projects/ps-epigen/users/biy022/biccn/data/SNAREdata")

In [3]:
matrix_dir <- "cell_peak_matrix/"
result_dir <- "regional_dar_deg/dar/"
metafile <- "cell_gene_matrix/20230313_RNA_metadata.xls"
meta_table <- read.table(metafile, header = TRUE, row.names = 1, sep = "\t")

In [5]:
for (file in dir_ls(matrix_dir, glob = "*rds")) {
    subclass <- path_ext_remove(path_file(file))
    print(subclass)
    if (subclass == "SST_CHODL") {
        next
    }
    
    subclass_result_dir <- sprintf("%s/%s/", result_dir, subclass)
    if (!dir_exists(subclass_result_dir)) {
        dir_create(subclass_result_dir)
    }
    else {
        next
    }
    subclass_matrix_file <- sprintf("%s/%s.rds", matrix_dir, subclass)
    subclass_matrix <- readRDS(subclass_matrix_file)
    subclass_meta <- meta_table[colnames(subclass_matrix), ]
    
    subclass_meta <- subclass_meta[match(colnames(subclass_matrix), rownames(subclass_meta)), ]
    subclass_meta$region_donor <- paste(subclass_meta$Region, subclass_meta$PatientID, sep = "_")
    transposed_matrix <- t(subclass_matrix)
    summed_counts <- rowsum(transposed_matrix, group = subclass_meta$region_donor)
    subclass_matrix_aggr <- t(summed_counts)
    
    group_info <- do.call(rbind, strsplit(colnames(subclass_matrix_aggr), "_"))
    group_info <- data.frame(group_info)
    colnames(group_info) <- c("region", "patient")
    rownames(group_info) <- colnames(subclass_matrix_aggr)
    
    group <- factor(group_info$region)
    y <- DGEList(counts = subclass_matrix_aggr, group = group, remove.zeros = TRUE)
    keep <- filterByExpr(y, min.count = 5, min.total.count = 15)
    y <- y[keep, , keep.lib.sizes=FALSE]
    y <- calcNormFactors(y)
    design <- model.matrix(~0+group, data = y$samples)
    colnames(design) <- levels(y$samples$group)
    y <- estimateDisp(y, design = design, robust = TRUE)
    fit <- glmQLFit(y, design = design)
    
    n_levels <- length(levels(y$samples$group))
    contrast_base <- rep(-(1.0 / (n_levels - 1)), n_levels)
    for (i in 1:n_levels) {
        contrast <- contrast_base
        contrast[i] <- 1
        level <- levels(y$samples$group)[i]
        qlf <- glmQLFTest(fit, contrast = contrast)
        result <- topTags(qlf, n = Inf, sort.by = "PValue")$table
        write.table(
            result, sprintf("%s/%s_result.tsv", subclass_result_dir, level), 
            col.names = TRUE, row.names = TRUE, sep="\t", quote = FALSE
        )
    }
}

[1] "Astro"
[1] "Chandelier"
[1] "Endo"
[1] "L2_3_IT"
[1] "L4_IT"
[1] "L5_6_NP"
[1] "L5_ET"
[1] "L5_IT"
[1] "L6B"
[1] "L6_CT"
[1] "L6_IT"
[1] "L6_IT_Car3"
[1] "LAMP5"
[1] "LAMP5_LHX6"
[1] "Micro"
[1] "OPC"
[1] "Oligo"
[1] "PAX6"
[1] "PVALB"
[1] "SNCG"
[1] "SST"
[1] "SST_CHODL"
[1] "VIP"


Removing 27 rows with all zero counts



[1] "VLMC"


Removing 36839 rows with all zero counts

